Attempt to implement a similar framework as explained in Csörnyei et al. (2023) for SNe type II to SNe type Ia. This means, estimate the time of explosion from the peak of the light curve and not from the early-light curve as done with the fireball fitting procedure. Clearly did not finish this... 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import scipy.stats
from ultranest import ReactiveNestedSampler
from ultranest.plot import cornerplot
from scipy.special import lambertw
import os
from scipy.optimize import curve_fit
import scipy.stats
import numpy
import pdb
import warnings
warnings.filterwarnings("ignore")

Load in the data, get rid of NaNs and separate the data into a list according to separate bands

In [47]:
sn='2020jhf'
lc_data = pd.read_csv('/Users/lalvopis/SNe_ICE/Photometry/geza_typeII/'+sn+'.csv', sep = ',')
cnd = np.isnan(lc_data['f'])

lc_data = lc_data[~cnd]

bands = ['c', 'o', 'ztfg', 'ztfr']
#bands = ['o', 'ztfg', 'ztfr']
lcs = []
for i in range (len(bands)):
    lcs.append(lc_data[lc_data['band'] == bands[i]])
    
#print(lcs)

Cut the relevant parts. Since the fitting function is simple, we can only use the initial rise, the first peak/plateu and the non-detections

In [48]:
cut_lcs = []

#2020jhf
c_lim = [58960, 59000]
o_lim = [58960, 59000]
g_lim = [58960, 59000]
r_lim = [58960, 59000]

for i in range (len(lcs)):
    if lcs[i]['band'].values[0] == 'c':
        lim_cond = (lcs[i]['mjd'] > c_lim[0]) & (lcs[i]['mjd'] < c_lim[1])
    if lcs[i]['band'].values[0] == 'o':
        lim_cond = (lcs[i]['mjd'] > o_lim[0]) & (lcs[i]['mjd'] < o_lim[1])
    elif lcs[i]['band'].values[0] == 'ztfg':
        lim_cond = (lcs[i]['mjd'] > g_lim[0]) & (lcs[i]['mjd'] < g_lim[1])
    else:
        lim_cond = (lcs[i]['mjd'] > r_lim[0]) & (lcs[i]['mjd'] < r_lim[1])
        
    cut_lcs.append(lcs[i][lim_cond])
    
print(cut_lcs)

[         mjd        f     ef band
18  58967.41     3.00   4.43    c
26  58983.35  1414.67   8.28    c
28  58987.43  1911.57   6.86    c
31  58995.40  2520.67  10.39    c,          mjd        f     ef band
16  58961.42    17.75   5.50    o
17  58965.43    29.75   5.88    o
19  58968.40   -17.75   5.00    o
20  58969.40    18.25   5.00    o
21  58971.39     4.00   7.89    o
22  58975.39    58.50  12.25    o
23  58977.44   191.50   8.50    o
24  58979.43   490.50   8.75    o
25  58981.39   911.25   7.00    o
27  58985.40  1800.00   8.50    o
29  58989.33  2198.75   8.50    o
30  58993.39  2212.50   9.25    o
32  58997.34  1969.75   9.25    o
33  58999.32  1763.50   9.50    o,               mjd            f         ef  band
1002  58962.23478     1.480905   3.581306  ztfg
1004  58963.28924     0.286161   1.954573  ztfg
1005  58963.28969    -1.007234   2.022928  ztfg
1006  58963.31265     2.557525   3.541900  ztfg
1007  58963.31509    -2.796298   3.323216  ztfg
1011  58965.25530    -4.53984

Separate actual detections and non-detections by putting in a limit. This is a very by-hand step; I just made this to follow the prescription of the past analysis

In [49]:
lcs_nd = []
photo_vls = []
for i in range (len(cut_lcs)):
    if len(cut_lcs[i]) > 0:
        cond = cut_lcs[i]['mjd'] < 58975
        lcs_nd.append(cut_lcs[i][cond].values[:,:3].astype(float))
        photo_vls.append(cut_lcs[i][~cond].values[:,:3].astype(float))
    else:
        lcs_nd.append([])
        photo_vls.append([])

In [50]:
# The initial t0 guess
t0_guess = 58974

# Normalize the flux values, subtract initial t0 guess from times.
for i in range (len(photo_vls)):
    if len(photo_vls[i]) > 0:
        norm_fact = max(photo_vls[i][:,1])
        lcs_nd[i][:,2] /= norm_fact
        lcs_nd[i][:,1] /= norm_fact
        lcs_nd[i][:,0] -= t0_guess
        
        photo_vls[i][:,2] /= norm_fact
        photo_vls[i][:,1] /= norm_fact
        photo_vls[i][:,0] -= t0_guess
        
print(lcs_nd)

[array([[-6.59000000e+00,  1.19015976e-03,  1.75746924e-03]]), array([[-1.25800000e+01,  8.02259887e-03,  2.48587571e-03],
       [-8.57000000e+00,  1.34463277e-02,  2.65762712e-03],
       [-5.60000000e+00, -8.02259887e-03,  2.25988701e-03],
       [-4.60000000e+00,  8.24858757e-03,  2.25988701e-03],
       [-2.61000000e+00,  1.80790960e-03,  3.56610169e-03]]), array([[-1.17652200e+01,  5.76452515e-04,  1.39404832e-03],
       [-1.07107600e+01,  1.11390190e-04,  7.60831251e-04],
       [-1.07103100e+01, -3.92072973e-04,  7.87438880e-04],
       [-1.06873500e+01,  9.95534214e-04,  1.37870901e-03],
       [-1.06849100e+01, -1.08847831e-03,  1.29358470e-03],
       [-8.74470000e+00, -1.76716549e-03,  1.22211521e-03],
       [-5.70098000e+00, -1.98441003e-03,  1.20956190e-03],
       [-2.70243000e+00,  3.66987645e-06,  2.15839804e-03],
       [ 2.07050000e-01, -8.41475465e-03,  4.11038400e-03]]), array([[-1.17228700e+01,  3.06382201e-03,  1.58454958e-03],
       [-9.70946000e+00,  2.16298

## Fitting procedure with UltraNest

For ultranest one has to set up the parameters and the priors in a reasonable range

In [51]:
parameters = ['t0', 'ac', 'ao', 'ag', 'ar', 'nc', 'no', 'ng', 'nr' ,'f']
#parameters = ['t0', 'fmo', 'fmg', 'fmr', 'teo', 'teg', 'ter' ,'f']


def prior_transform(cube):
    # the argument, cube, consists of values from 0 to 1
    # we have to convert them to physical scales
    
    params = cube.copy()
    params[0] = cube[0] * 10 + 5      # parameter for t0; t0 guess has already been subtracted from the data
    params[1] = cube[1] * 0.6 + 4   # the scaling value
    params[2] = cube[2] * 0.6 + 4
    params[3] = cube[3] * 0.6 + 4
    params[4] = cube[4] * 0.6 + 4
    params[5] = cube[5] * 0.6 + 2           # the expoent, it should be around 2
    params[6] = cube[6] * 0.6 + 2
    params[7] = cube[7] * 0.6 +2
    params[8] = cube[8] * 0.6 +2
    params[9] = cube[9] * 3           # error inflation parameter; in case uncertainties are underestimated
    
    return params

In [52]:
# The fitting function; fireball model - power law fitting
def flux_evol(x, a, t0, n):    
    return a * (x - t0)**n

Likelihood for the Bayesian fitting: this is where the different band light curves get connected through t0 and the rise time condition

In [53]:
def log_likelihood(params):
    
    t0, ac, ao, ag, ar, nc, no, ng, nr, f = params
    alphas = [ac, ao, ag, ar] #scaling factor for each band
    ns = [nc, no, ng, nr] #expoent for each band

    # Check for the models in each band if the model light curve goes ABOVE the non-detections
    #if yes, that is a bad model
    for i in range (len(photo_vls)):   # I have put in here the last g_nondet for simplicity
        try:
            # It is enough to check this for the last non-det, if it is deep enough
            non_dec_cond = lcs_nd[i][-1,1] < flux_evol(lcs_nd[i][-1,0], a=alphas[i], t0=t0, n=ns[i])  

            if non_dec_cond:
                return -9999
            
        # It can happen that we have no non-detections, then we cannot check the condition
        #this is here to catch that
        except (IndexError,AttributeError):  
            pass

    # if any of the above, return -9999, force the sampler away
    # if we are still in here, do the actual loglike calculation
    loglike = 0
    for i in range (len(photo_vls)):
        y_model = flux_evol(photo_vls[i][:,0], a=alphas[i], t0=t0, n=ns[i])
    
        sigma2 = photo_vls[i][:,2]**2 + y_model **2 * f**2    # the error inflation comes in here
        loglike += -0.5 * np.sum((y_model - photo_vls[i][:,1])**2 / sigma2 + np.log(2*np.pi*sigma2))
        
        pdb.set_trace()

    return loglike

Do the fitting; this code can throw an AssertionError, if so, check if there are NaNs in the data.

If no, just run the above def scripts again, and try once more; sometimes it starts from the wrong initial parameters

In [54]:
sampler = ReactiveNestedSampler(parameters, log_likelihood, prior_transform,
    wrapped_params=np.full(len(parameters), False)
)

#400 does the trick, bcut if you need sharp posteriors, might want to increase it; computationally expensive!
result = sampler.run(min_num_live_points=400, dKL=np.inf, min_ess=100) 

> /var/folders/hb/hpll1ddj15bf35zghh_jv_0w00098x/T/ipykernel_99924/772349429.py(25)log_likelihood()
     23     # if we are still in here, do the actual loglike calculation
     24     loglike = 0
---> 25     for i in range (len(photo_vls)):
     26         y_model = flux_evol(photo_vls[i][:,0], a=alphas[i], t0=t0, n=ns[i])
     27 

ipdb> alphas
[4.072677175187902, 4.560404468039326, 4.070090774756253, 4.56163052514792]
ipdb> ns
[2.307131044077833, 2.4010683688083203, 2.154539960121755, 2.2422345037758964]
ipdb> photo_vls[i][:,0]
array([ 9.35, 13.43, 21.4 ])
ipdb> t0
8.011513064783177
ipdb> c
> /var/folders/hb/hpll1ddj15bf35zghh_jv_0w00098x/T/ipykernel_99924/772349429.py(25)log_likelihood()
     23     # if we are still in here, do the actual loglike calculation
     24     loglike = 0
---> 25     for i in range (len(photo_vls)):
     26         y_model = flux_evol(photo_vls[i][:,0], a=alphas[i], t0=t0, n=ns[i])
     27 

ipdb> c
> /var/folders/hb/hpll1ddj15bf35zghh_jv_0w00098x/T/ipyk

ipdb> c


AssertionError: Error in loglikelihood function: returned non-finite number: [nan nan] for input u=[[0.30115131 0.12112863 0.93400745 0.11681796 0.93605088 0.51188507
  0.66844728 0.2575666  0.40372417 0.56560131]
 [0.31278216 0.13329834 0.23363154 0.53428986 0.55814863 0.74320432
  0.10832185 0.88049953 0.90525884 0.4619362 ]] p=[[8.01151306 4.07267718 4.56040447 4.07009077 4.56163053 2.30713104
  2.40106837 2.15453996 2.2422345  1.69680392]
 [8.12782165 4.079979   4.14017893 4.32057392 4.33488918 2.44592259
  2.06499311 2.52829972 2.54315531 1.38580859]]